In [4]:
import scipy.io
import pandas as pd

# Load the .mat file
mat_data = scipy.io.loadmat('../data/InSAR_vertical_wholeCV_downsample500mx500m.mat')

# Print the contents of the .mat file
DZ = mat_data['DZ'].flatten()  # Convert to 1D array
Z = mat_data['Z'].flatten()    # Convert to 1D array
lnx = mat_data['lnx'].flatten()  # Convert to 1D array
lty = mat_data['lty'].flatten()  # Convert to 1D array

# Create a DataFrame
df = pd.DataFrame({
    'Longitude': lnx,
    'Latitude': lty,
    'Z': Z,
    'DZ': DZ
})

In [5]:
df

,Longitude,Latitude,Z,DZ
0,-122.541102,39.859374,0.566691,0.069111
1,-122.540790,39.863396,0.776471,0.064225
2,-122.536195,39.855112,0.510672,0.061217
3,-122.535884,39.859134,0.194299,0.078462
4,-122.535572,39.863156,0.520263,0.056262
...,...,...,...,...
265323,-118.606756,35.283054,0.542738,0.101017
265324,-118.606143,35.286905,0.641504,0.082334
265325,-118.605529,35.290757,0.753156,0.089112
265326,-118.604915,35.294608,0.603030,0.098666


In [6]:
len(df.Longitude.unique())

265328